## Benchmarks
This demo shows the performance of other SOTA graph embeddings methods and their limitations:
they do not take attributes into account (only can handle discreet attributes)
they are very dependable onto the reinitialization of the random walks and minor graph changes

In [1]:
import os,sys
import random
import numpy as np
import networkx as nx

sys.path.append(os.path.realpath('lib'))
from lib.data_loader import load_local_data
from benchmarks.sub2vec import Sub2vec

## sub2vec demo

In this approach the authours were inspired by the paragraph2vec method of paragrahps embeddings. Creates sub-graph embeddings which takes neighborhood & structural information (separately) into account. The neighborhood info is similar to node2vec. The structural info is described using random walks, where each node is represented as a ratio of node degree to the size of  a subgraph, the graph-paths track how the density of edges changes in a neighbourhood. The nodes are coded (depending on their degree/graph size ratio) into a log scale vocabulary. To learn the final embeddings, researchers propose two methods: sub2vec-DM (predicting neigh node and graph) and sud2vec-DBON (co-occurence based learning). Negative samling is used as in word2vec.

In [55]:
dataset_n='aids' # the IGN  and BZR are the alternative datasets
path='data/'
X,y=load_local_data(path,dataset_n, attributes=False, use_node_deg=False)
sub2vec = Sub2vec(property='s', walkLength=1000, output='aids_walk', d=128, iter=100, windowSize=2, p=0.5, model='dm')
# the parameters chosen are the ones from the paper, I only modified the walklenght for a smalled dataset
sub2vec.obtainRandomWalks(X)
embeddings = sub2vec.calculateEmbeddings()
print(embeddings.shape)

Total vects  2000
(2000, 128)


In [3]:
# re-calculate the embeddings and demonstrate that the cosine similarity doesn't work within rounds 

In [56]:
sub2vec.obtainRandomWalks(X)
embeddings2 = sub2vec.calculateEmbeddings()


Total vects  2000


In [61]:
# display cosine similarity for first 10 embeddings
from sklearn.metrics.pairwise import cosine_similarity
num_graphs, d = embeddings.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs within rounds is {cosine_similarity(embeddings[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")


Similarity of two graphs within rounds is [[0.06005514]]
Similarity of two graphs within rounds is [[0.13349625]]
Similarity of two graphs within rounds is [[0.09431593]]
Similarity of two graphs within rounds is [[0.05708874]]
Similarity of two graphs within rounds is [[-0.0155139]]
Similarity of two graphs within rounds is [[0.17029813]]
Similarity of two graphs within rounds is [[0.12372799]]
Similarity of two graphs within rounds is [[-0.03390585]]
Similarity of two graphs within rounds is [[0.16328917]]
Similarity of two graphs within rounds is [[0.24532312]]


In [34]:
# function to calculate all the values and display stats
def meanstd_similarity(emb1, emb2):
    res_similarity = []
    num_graphs, d = emb1.shape
    for i in range(num_graphs):
        res_similarity.append(cosine_similarity(emb1[i,:].reshape(1, -1), emb2[i,:].reshape(1, -1))[0][0])
    print(f"Mean cosine similarity is {np.mean(res_similarity)}, std is {np.std(res_similarity)} for the pairwise comparison if {num_graphs} graphs")  


In [35]:
meanstd_similarity(emb1 = embeddings, emb2 = embeddings2)

Mean cosine similarity is 0.11877530068159103, std is 0.15287145972251892 for the pairwise comparison if 2000 graphs


In [33]:
# and now calculate the embeddings in one go and check whether they are similar or not this way

In [25]:
X_double = np.hstack((X,X))
sub2vec.obtainRandomWalks(X_double)
embeddings1_2 = sub2vec.calculateEmbeddings()

Total vects  4000


In [36]:
embeddings1 = embeddings1_2[:2000,:]
embeddings2 = embeddings1_2[2000:,:]
num_graphs, d = embeddings1.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs from the same round is {cosine_similarity(embeddings1[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")
# much better but still very small at times, not consistent and these are the exactly same graphs!

Similarity of two graphs from the same round is [[0.9857781]]
Similarity of two graphs from the same round is [[0.98320913]]
Similarity of two graphs from the same round is [[0.9268111]]
Similarity of two graphs from the same round is [[0.95995694]]
Similarity of two graphs from the same round is [[0.9697628]]
Similarity of two graphs from the same round is [[0.8995649]]
Similarity of two graphs from the same round is [[0.93944114]]
Similarity of two graphs from the same round is [[0.9864575]]
Similarity of two graphs from the same round is [[0.94997174]]
Similarity of two graphs from the same round is [[0.9463763]]


In [37]:
meanstd_similarity(emb1 = embeddings1, emb2 = embeddings2)

Mean cosine similarity is 0.9380630850791931, std is 0.10485057532787323 for the pairwise comparison if 2000 graphs


In [69]:
# I modified the graphs removing one single (always connected only to one or max two neighbors node)

In [38]:
from copy import deepcopy
def remove_single_node(graphs_list):
    ''' function just removes a random node from a graph which has one or two neighbors'''
    modified_graphs = []
    for graph in graphs_list:
        nx_graph = graph.nx_graph # get the graph
        degree = nx_graph.degree
        degreeDict = dict(degree)
        random_nodes_to_delete = []
        # pick nodes which have 1 neighbor only
        for node, degree in degreeDict.items():
            if degree == 1:
                random_nodes_to_delete.append(node)
        # else pick nodes which have 2 neighbors
        if len(random_nodes_to_delete) == 0:
            for node, degree in degreeDict.items():
                if degree == 2:
                    random_nodes_to_delete.append(node)
        if len(random_nodes_to_delete)==0:
            modified_graphs.append(graph) # leave the graph as it was
            print('A graph which has no nodes with degrees 1 & 2 detected')
        else:
            copy_graph = deepcopy(graph) # create a new deep copy
            copy_graph.nx_graph.remove_node(random.choice(random_nodes_to_delete))

            modified_graphs.append(copy_graph)
        #
    return modified_graphs


In [39]:
# a small unit test of a function above
subset_X = X[:10]
modified_subset = remove_single_node(subset_X)
print('done')
for i in range(10):
    print(f"graph before modif has {len(subset_X[i].nx_graph)} and after modif {len(modified_subset[i].nx_graph)}")


done
graph before modif has 47 and after modif 46
graph before modif has 11 and after modif 10
graph before modif has 9 and after modif 8
graph before modif has 10 and after modif 9
graph before modif has 16 and after modif 15
graph before modif has 9 and after modif 8
graph before modif has 11 and after modif 10
graph before modif has 10 and after modif 9
graph before modif has 17 and after modif 16
graph before modif has 9 and after modif 8


And now check if the resulting embeddings are different

In [40]:
X_modified =  remove_single_node(X)
X_original_and_modidied = np.hstack((X,X_modified))
sub2vec.obtainRandomWalks(X_original_and_modidied)
embeddings1_2 = sub2vec.calculateEmbeddings()

Total vects  4000


In [41]:
embeddings1 = embeddings1_2[:2000,:]
embeddings2 = embeddings1_2[2000:,:]
num_graphs, d = embeddings1.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs with a single node removed is {cosine_similarity(embeddings1[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")
# sometimes the change is significant.

Similarity of two graphs with a single node removed is [[0.45863032]]
Similarity of two graphs with a single node removed is [[0.86961746]]
Similarity of two graphs with a single node removed is [[0.9778628]]
Similarity of two graphs with a single node removed is [[0.8524918]]
Similarity of two graphs with a single node removed is [[0.9028555]]
Similarity of two graphs with a single node removed is [[0.97142464]]
Similarity of two graphs with a single node removed is [[0.92181873]]
Similarity of two graphs with a single node removed is [[0.9858958]]
Similarity of two graphs with a single node removed is [[0.8797715]]
Similarity of two graphs with a single node removed is [[0.9812438]]


In [42]:
meanstd_similarity(emb1 = embeddings1, emb2 = embeddings2)

Mean cosine similarity is 0.8966434597969055, std is 0.1476893275976181 for the pairwise comparison if 2000 graphs


### Conclusion Sub2Vec
The results are not too bad, however, for some cases it is still a lot for the case when we need to compare the vectors. A complete disaster between runs - the embeddings are not coherent at all. Affected quite a lot by the random walk. 
And the main problem with this method - we cannot really use the attributes which are [potentially] extremely important.

# Graph2Vec demo

A method representing entire graphs as fixed length feature vectors - inspired by  neural document embedding models, the authors extend the same to learn graph embeddings.graph2vec proposes to view an entire graph as a document and the rooted subgraphs around every node in the graph as words that compose the document and extend document embedding neural networks to learn representations of entire graphs. An input of algorithms is a set of labeled or unlabeled graphs, in the case of unlabeled graphs  nodes are labeled  with their degree. In graph2vec,the are graphs analogical to documents
that are composed of rooted subgraphs which, in turn, are
analogical words from a special language and extend document embedding models to learn graph embeddings. The main motivation behind this idea is that structurally similar graphs will be close to each other in the embedding space. 

Algorithm: 

input -   Set of graphs

1) extract rooted subgraphs and assign a unique
label for all the rooted subgraphs in the vocabulary 

2) these hashed features (tags ( Weisfeiler-Lehman kernel decomposes a graph into subtrees ->https://www.cse.wustl.edu/~muhan/papers/KDD_2017.pdf) and node degrees (words) describe each sub-graph 

3) train a doc2vec skip-gram model using negative sampling. DBOW model is used.

In [43]:
import hashlib
import tqdm
from joblib import Parallel, delayed
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from benchmarks.graph2vec import feature_extractor # I modified the original code by the authors to directly use it on our data

In [ ]:
dataset_n = 'aids'
path = 'data/'
X, y = load_local_data(path, dataset_n, attributes=False, use_node_deg=False)
X = list(X)
print("\nFeature extraction started.\n")
document_collections = Parallel(n_jobs=1)(delayed(feature_extractor)(g,2, str(i)) for i, g in enumerate(X))
print("\nOptimization started.\n")

model = Doc2Vec(document_collections, vector_size = 128, window = 2, min_count = 5, dm = 0,
                sample = 0.0001, workers = 10, epochs = 20, alpha =0.025)
embeddings = model.docvecs.vectors_docs
print(f"Returned embeddings shape is {embeddings.shape}")

In [45]:
document_collections2 = Parallel(n_jobs=1)(delayed(feature_extractor)(g,2, str(i)) for i, g in enumerate(X))
model = Doc2Vec(document_collections2, vector_size = 128, window = 2, min_count = 5, dm = 0,
                sample = 0.0001, workers = 10, epochs = 20, alpha =0.025)
embeddings2 = model.docvecs.vectors_docs

Check the similarity within the runs

In [46]:
num_graphs, d = embeddings.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs within rounds is {cosine_similarity(embeddings[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")


Similarity of two graphs within rounds is [[0.848712]]
Similarity of two graphs within rounds is [[0.8972695]]
Similarity of two graphs within rounds is [[0.8492506]]
Similarity of two graphs within rounds is [[0.8924836]]
Similarity of two graphs within rounds is [[0.77217966]]
Similarity of two graphs within rounds is [[0.7804389]]
Similarity of two graphs within rounds is [[0.87400985]]
Similarity of two graphs within rounds is [[0.65968746]]
Similarity of two graphs within rounds is [[0.8078866]]
Similarity of two graphs within rounds is [[0.8581012]]


In [47]:
meanstd_similarity(emb1 = embeddings, emb2 = embeddings2)

Mean cosine similarity is 0.8272595405578613, std is 0.05931011587381363 for the pairwise comparison if 2000 graphs


In [ ]:
# Similarity in a  one run within the same graphs

In [48]:
X_double = np.hstack((X,X))
document_collections = Parallel(n_jobs=1)(delayed(feature_extractor)(g,2, str(i)) for i, g in enumerate(X_double))
model = Doc2Vec(document_collections, vector_size = 128, window = 2, min_count = 5, dm = 0,
                sample = 0.0001, workers = 10, epochs = 20, alpha =0.025)
embeddings1_2 = model.docvecs.vectors_docs

In [50]:
embeddings1 = embeddings1_2[:2000,:]
embeddings2 = embeddings1_2[2000:,:]
num_graphs, d = embeddings1.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs from the same round is {cosine_similarity(embeddings1[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")
# this actually looks pretty good! So the WL labeling gives pretty good results in comparison with random walks 

Similarity of two graphs from the same round is [[0.9925771]]
Similarity of two graphs from the same round is [[0.9945583]]
Similarity of two graphs from the same round is [[0.9765564]]
Similarity of two graphs from the same round is [[0.9957062]]
Similarity of two graphs from the same round is [[0.98460644]]
Similarity of two graphs from the same round is [[0.9907874]]
Similarity of two graphs from the same round is [[0.98394835]]
Similarity of two graphs from the same round is [[0.9813585]]
Similarity of two graphs from the same round is [[0.9954908]]
Similarity of two graphs from the same round is [[0.98716354]]


In [51]:
meanstd_similarity(emb1 = embeddings1, emb2 = embeddings2)

Mean cosine similarity is 0.9899162650108337, std is 0.01579388417303562 for the pairwise comparison if 2000 graphs


Finally, check if the algo is robust for node removal 

In [52]:
X_modified =  remove_single_node(X)
X_original_and_modidied = np.hstack((X,X_modified))
document_collections = Parallel(n_jobs=1)(delayed(feature_extractor)(g,2, str(i)) for i, g in enumerate(X_original_and_modidied))
model = Doc2Vec(document_collections, vector_size = 128, window = 2, min_count = 5, dm = 0,
                sample = 0.0001, workers = 10, epochs = 20, alpha =0.025)
embeddings1_2 = model.docvecs.vectors_docs


In [53]:
embeddings1 = embeddings1_2[:2000,:]
embeddings2 = embeddings1_2[2000:,:]
num_graphs, d = embeddings1.shape
random_graphs =  np.random.randint(0, num_graphs, size=(10))
for i in random_graphs:
    print(f"Similarity of two graphs from the same round is {cosine_similarity(embeddings1[i,:].reshape(1, -1), embeddings2[i,:].reshape(1, -1))}")
# this is no longer good

Similarity of two graphs from the same round is [[0.6321049]]
Similarity of two graphs from the same round is [[0.8713942]]
Similarity of two graphs from the same round is [[0.932484]]
Similarity of two graphs from the same round is [[0.8647433]]
Similarity of two graphs from the same round is [[0.7991464]]
Similarity of two graphs from the same round is [[0.72401214]]
Similarity of two graphs from the same round is [[0.6373059]]
Similarity of two graphs from the same round is [[0.9049396]]
Similarity of two graphs from the same round is [[0.84791726]]
Similarity of two graphs from the same round is [[0.8490903]]


In [54]:
meanstd_similarity(emb1 = embeddings1, emb2 = embeddings2)

Mean cosine similarity is 0.7509998679161072, std is 0.14412258565425873 for the pairwise comparison if 2000 graphs


Not really robust to this one, so this is a limitation of graph2vec.

# Conclusion on two embedding methods

It will be awesome to allow to use node attributes of graphs in resulting embeddings. None of this algo uses weights of the edges either. Node2vec in the walk generation. 

It will be nice to have constant embeddings withing algorithm run but in the same moment to make them robust for minor graph 
changes. There are methods which allow for that (for example, subgraph mathcing kernel) - but they way too long to compute for a real-case scenario.
If we could have had an embedding having better properties than the existing ones - it can be very interesting for some applications.


# Node2Vec

In [60]:
from benchmarks.node2vec import node2vec

This approach is very similar to word2vec embedding and is a direct improuvement of DeepWalk. The nodes are embedded based on their proximity and inter-connection, so connected node results in similar embeddings. The new idea in node2vec is the new sampling strategy allowing to explore the node neighborhood.

The algorithm will take a big graph as an input, where nodes have unique labels. 
Then each node will be embedded and assigned a key-word (it's ID), and we can retrieve the embedding of the node using the ID.

A demo on node2vec can be found here [https://github.com/eliorc/Medium/blob/master/Nod2Vec-FIFA17-Example.ipynb], and I tried a small one on the Geo data I have.

In [64]:
super_graph = nx.read_gpickle('data/supergraph_moselle_vector.gpickle')
print(f'this graph has {len(super_graph)} nodes')

this graph has 18913 nodes


In [67]:
colors =np.repeat('m', len(super_graph.edges))
for i, edge in enumerate(super_graph.edges(data=True)):
    if edge[2]['nature']==1:
        colors[i] = 'b'

fig, ax = plt.subplots(figsize=(20.0, 20.0))        
nx.draw(super_graph,node_color='#A0CBE2',edge_color = colors,width=4,edge_cmap=plt.cm.Blues,with_labels=True)

SyntaxError: invalid character in identifier (<ipython-input-67-46e557a7146b>, line 6)